In [1]:
import hashlib
import time
import json


class Block:
    def __init__(self, index, previous_hash, timestamp, data):
        """
        A block contains:
        - Index: Position of the block in the chain.
        - Previous hash: Hash of the previous block for linking.
        - Timestamp: Time the block was created.
        - Data: Contains the voter ID and vote choice.
        """
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        """
        Calculates the hash of the block using its contents.
        """
        block_content = f"{self.index}{self.previous_hash}{self.timestamp}{json.dumps(self.data)}"
        return hashlib.sha256(block_content.encode()).hexdigest()


class Blockchain:
    def __init__(self):
        """
        Initializes the blockchain with a genesis block.
        """
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        """
        Creates the first block of the chain.
        """
        return Block(0, "0", time.time(), {"message": "Genesis Block"})

    def get_latest_block(self):
        """
        Retrieves the latest block in the chain.
        """
        return self.chain[-1]

    def add_block(self, new_block):
        """
        Adds a new block to the chain after validation.
        """
        new_block.previous_hash = self.get_latest_block().hash
        new_block.hash = new_block.calculate_hash()
        self.chain.append(new_block)

    def is_chain_valid(self):
        """
        Validates the blockchain by checking the hashes and links.
        """
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            # Validate current block hash
            if current_block.hash != current_block.calculate_hash():
                return False

            # Validate link to the previous block
            if current_block.previous_hash != previous_block.hash:
                return False

        return True


class VotingSystem:
    def __init__(self):
        self.blockchain = Blockchain()
        self.voters = set()  # To track voters and prevent double voting
        self.candidates = ["Candidate A", "Candidate B", "Candidate C"]  # List of candidates

    def cast_vote(self, voter_id, candidate):
        """
        Adds a vote as a block to the blockchain.
        """
        if voter_id in self.voters:
            return f"Voter {voter_id} has already voted."

        if candidate not in self.candidates:
            return f"Invalid candidate. Valid options are: {', '.join(self.candidates)}."

        timestamp = time.time()
        vote_data = {"voter_id": voter_id, "candidate": candidate}
        new_block = Block(len(self.blockchain.chain), self.blockchain.get_latest_block().hash, timestamp, vote_data)
        self.blockchain.add_block(new_block)
        self.voters.add(voter_id)
        return f"Vote cast successfully for {candidate} by Voter {voter_id}."

    def display_votes(self):
        """
        Displays all votes stored in the blockchain.
        """
        print("\n--- Voting Records ---")
        for block in self.blockchain.chain:
            print(f"Block {block.index}:")
            print(f"Timestamp: {time.ctime(block.timestamp)}")
            print(f"Data: {block.data}")
            print(f"Hash: {block.hash}")
            print(f"Previous Hash: {block.previous_hash}")
            print()

    def validate_votes(self):
        """
        Validates the integrity of the voting records.
        """
        return self.blockchain.is_chain_valid()


def main():
    voting_system = VotingSystem()
    while True:
        print("\n--- Blockchain Voting System ---")
        print("1. Cast Vote")
        print("2. Display Votes")
        print("3. Validate Blockchain")
        print("4. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            voter_id = input("Enter your voter ID: ")
            print(f"Candidates: {', '.join(voting_system.candidates)}")
            candidate = input("Enter the candidate you want to vote for: ")
            result = voting_system.cast_vote(voter_id, candidate)
            print(result)

        elif choice == "2":
            voting_system.display_votes()

        elif choice == "3":
            if voting_system.validate_votes():
                print("Blockchain is valid. All votes are secure.")
            else:
                print("Blockchain validation failed! Votes may be compromised.")

        elif choice == "4":
            print("Exiting the system. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")


if __name__ == "__main__":
    main()



--- Blockchain Voting System ---
1. Cast Vote
2. Display Votes
3. Validate Blockchain
4. Exit
Enter your choice: 1
Enter your voter ID: 548562
Candidates: Candidate A, Candidate B, Candidate C
Enter the candidate you want to vote for: Candidate A
Vote cast successfully for Candidate A by Voter 548562.

--- Blockchain Voting System ---
1. Cast Vote
2. Display Votes
3. Validate Blockchain
4. Exit
Enter your choice: 2

--- Voting Records ---
Block 0:
Timestamp: Tue Feb 18 13:09:13 2025
Data: {'message': 'Genesis Block'}
Hash: 5b4d70068708075b9f70d198f00c8fddabaa93ab91aff6e7163b5b187ae39272
Previous Hash: 0

Block 1:
Timestamp: Tue Feb 18 13:10:18 2025
Data: {'voter_id': '548562', 'candidate': 'Candidate A'}
Hash: 75b734bad70dd0a9c9e53ea0be5326bb89dded47b09499cd3ca078d8d1e4bf46
Previous Hash: 5b4d70068708075b9f70d198f00c8fddabaa93ab91aff6e7163b5b187ae39272


--- Blockchain Voting System ---
1. Cast Vote
2. Display Votes
3. Validate Blockchain
4. Exit
Enter your choice: 3
Blockchain is val